In [1]:
import os

import numpy as np

os.chdir('/Users/edwardchiu/Desktop/projects/baha-anime-analysis')
os.getcwd()

'/Users/edwardchiu/Desktop/projects/baha-anime-analysis'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import matplotlib.pyplot as plt
import ast


In [ ]:
df_anime = pd.read_csv('data/all_anime.csv')
df_anime.head()

In [ ]:
df_anime['types'] = df_anime['types'].apply(ast.literal_eval)

In [ ]:
df_anime['first_launched_date'] = pd.to_datetime(df_anime['first_launched_date'])
df_anime['first_launched_date'] = df_anime['first_launched_date'].astype(int)

In [ ]:
df_anime['total_view'] = df_anime['total_view'].fillna(df_anime['total_view'].min())
df_anime['score'] = df_anime['score'].fillna(df_anime['score'].min())

In [ ]:
scaler = StandardScaler()
df_anime[['scaled_launch', 'scaled_score']] = (
    scaler.fit_transform(df_anime[['first_launched_date', 'score']]))
df_anime[['scaled_launch', 'total_view', 'scaled_score']].describe()

In [ ]:
plt.hist(np.exp(df_anime['scaled_score']))
plt.show()

In [ ]:
plt.hist(np.exp(df_anime['scaled_launch']))
plt.show()

In [ ]:
plt.hist(np.log(df_anime['total_view']))
plt.show()

In [ ]:
df_anime['scaled_launch'] = np.exp(df_anime['scaled_launch'])
df_anime['scaled_score'] = np.exp(df_anime['scaled_score'])
df_anime['scaled_view'] = np.log(df_anime['total_view'])
df_anime[['scaled_launch', 'scaled_view', 'scaled_score']].describe()

In [ ]:
scaler = MinMaxScaler()
df_anime[['scaled_launch', 'scaled_view', 'scaled_score']] = scaler.fit_transform(
    df_anime[['scaled_launch', 'scaled_view', 'scaled_score']])
df_anime[['scaled_launch', 'scaled_view', 'scaled_score']].describe()

In [ ]:
df_anime[['scaled_launch2', 'scaled_view2', 'scaled_score2']] = scaler.fit_transform(
    df_anime[['first_launched_date', 'total_view', 'score']])
df_anime[['scaled_launch2', 'scaled_view2', 'scaled_score2']].describe()

In [ ]:
plt.hist(df_anime['scaled_launch2'])
plt.show()

In [ ]:
plt.hist(df_anime['scaled_score2'])
plt.show()

In [ ]:
plt.hist(df_anime['scaled_view2'])
plt.show()

In [ ]:
plt.hist(df_anime['total_view'])
plt.show()

In [ ]:
# Function to compute Jaccard similarity
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

In [ ]:
# Create an empty list to store the results
similarity_matrix = np.zeros((len(df_anime), len(df_anime)))
similarity_matrix

In [ ]:
for r in range(len(df_anime)):
    for c in range(r + 1):
        if r == c:
            similarity_matrix[r, c] = float('-inf')
        else:
            similarity = jaccard_similarity(set(df_anime.iloc[r]['types']), set(df_anime.iloc[c]['types']))
            similarity_matrix[r, c] = similarity
            similarity_matrix[c, r] = similarity

In [ ]:
similarity_df = pd.DataFrame(similarity_matrix, index=df_anime['name'], columns=df_anime['name'])

In [ ]:
# Specify the anime for which you want to find the top 5 similar animes
target_anime = ['鬼滅之刃 竈門炭治郎 立志篇', '咒術迴戰']
parameters = ['scaled_score']

# Get the row corresponding to the target anime
target_similarities = similarity_df.loc[:, target_anime]

scaled_metrics = df_anime.loc[:, ['name', 'scaled_launch', 'scaled_view', 'scaled_score']]
scaled_metrics = scaled_metrics.set_index('name')

target_similarities.loc[:, ['scaled_launch', 'scaled_view', 'scaled_score']] = scaled_metrics

target_similarities['main_metic'] = target_similarities[target_anime + parameters].mean(axis=1)

# Exclude the similarity with itself (which is always 1.0)
# target_similarities = target_similarities.drop(target_anime)
# 
# Sort the similarities in descending order and get the top 5 most similar animes
top_5_similar_animes = target_similarities.sort_values('main_metic', ascending=False).head(5)

# Print the top 5 similar animes
top_5_similar_animes


In [ ]:
top_5_similar_animes.index

In [ ]:
similarity_df[target_anime]

In [ ]:
pd.DataFrame(similarity_matrix[:, target_index],
             index=df_anime['name'], columns=target_anime)

In [ ]:
top_5_similar_animes

In [ ]:
df_anime[df_anime['name'] == '格萊普尼爾']

In [ ]:
df_anime['first_launched_date'] = pd.to_datetime(df_anime['first_launched_date'])

In [ ]:
df_anime['first_launched_date'].astype(int)

In [ ]:
import ast

In [ ]:
ast.literal_eval(df_anime.iloc[0]['types'])

In [ ]:
df_anime['types'].apply(lambda x: '、'.join(x))

In [2]:
from modules.recommend import AnimeRecommend

In [3]:
ar = AnimeRecommend()

In [20]:
animes = ar.anime_recommend(target_anime=['咒術迴戰'], parameters=['scaled_view'])

In [21]:
list(animes[:6])

['=HYPERLINK("https://ani.gamer.com.tw/animeRef.php?sn=113281", "咒術迴戰 第二季")',
 '=HYPERLINK("https://ani.gamer.com.tw/animeRef.php?sn=98149", "鬼滅之刃 竈門炭治郎 立志篇")',
 '=HYPERLINK("https://ani.gamer.com.tw/animeRef.php?sn=113154", "我想成為影之強者！")',
 '=HYPERLINK("https://ani.gamer.com.tw/animeRef.php?sn=113513", "鬼滅之刃 柱訓練篇")',
 '=HYPERLINK("https://ani.gamer.com.tw/animeRef.php?sn=113370", "不死不運")',
 '=HYPERLINK("https://ani.gamer.com.tw/animeRef.php?sn=93384", "JOJO 的奇妙冒險 黃金之風")']

In [22]:
# Update cells C2:H2 with the first six elements
ar.worksheet.update(range_name='B6:G6', values=[list(animes[:6])], raw=False)

ar.worksheet.update(range_name='B11:G11', values=[list(animes[6:])], raw=False)

{'spreadsheetId': '1F94CV-TTa628TumABt3DOF_beqJxQTJ-Mjp1nHkWQDE',
 'updatedRange': "'Anime Recommendation'!B11:G11",
 'updatedRows': 1,
 'updatedColumns': 6,
 'updatedCells': 6}

In [ ]:
animes[:6]